![alt text](DataKind_orange.png)

# Omidyar Extractives Project 1
## Extract Contract Text (Notebook 1 of 5)
### 1. Extracts contract text using urls to pdf documents contained in contract metadata file downloaded from resourcecontracts.org/contracts
### 2. Appends contract text to original contract metadata file and pickles dataframe for analysis

In [19]:
import codecs
import urllib
import urllib2
import pandas as pd
from tqdm import tqdm
tqdm.pandas(desc="Progress")

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO

In [20]:
contracts = pd.read_csv('contract_data/contract_2017-06-21.csv')
outfile = 'contract_data/1_contracts_with_text.pkl'

In [21]:
def pdf_from_url_to_txt(url):
    '''
    Takes a url to a pdf document and returns the text as a string
    '''
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    # Open the url provided as an argument to the function and read the content
    f = urllib2.urlopen(urllib2.Request(url)).read()
    # Cast to StringIO object
    fp = StringIO(f)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()
    for page in PDFPage.get_pages(fp,
                                  pagenos,
                                  maxpages=maxpages,
                                  password=password,
                                  caching=caching,
                                  check_extractable=False):
        interpreter.process_page(page)
    fp.close()
    device.close()
    str = retstr.getvalue()
    retstr.close()
    return str

In [22]:
def get_contract_text(url):
    '''
    Takes url to a pdf document and returns text as a string
    '''
    if url[-7:] == '....pdf':
        txt_file = url[:-6] + 'txt' 
    else:
        txt_file = url[:-3] + 'txt'
    text = urllib.urlopen(txt_file).read()
    if 'AccessDenied' in text[0:1000]:
        text = pdf_from_url_to_txt(url)
    return text

In [ ]:
contracts['Contract_Text'] = contracts['Pdf Url'].progress_apply(get_contract_text)

In [ ]:
print "Number of contracts extracted: " + str(len(contracts))

In [ ]:
contracts.to_pickle(outfile)

## Historical Contracts

In [ ]:
### Directly downloaded contract files
historical_files = 'contract_data/Contracts_Historical/'

### Contract Backlog

In [ ]:
backlog_files = 'contract_data/Contracts_Backlog/'

### Files from Colobmian website

In [13]:
### Import list of Colombian oil contract pdf links
f = codecs.open('contract_data/Contracts_Backlog/colombia_oil_contracts.txt','r', encoding='utf-8').readlines()

In [24]:
colombian_backlog = pd.DataFrame()
for url in f[0:3]:
    print url
    colombian_backlog = colombian_backlog.append([url,pdf_from_url_to_txt(url)])

http://www.anh.gov.co/Asignacion-de-areas/Contratos%20EP%20y%20TEAS%20firmados/CTO EyP BUENAVISTA 2004.pdf



HTTPError: HTTP Error 400: Bad Request

### Other contract file links

In [ ]:
other_contracts = ['https://wikileaks.org/car-mining/html/PTI-IAS-CAR-Contract/PTI-IAS-CAR-Contract.pdf',
'https://wikileaks.org/car-mining/html/DIG-CAR-Contract/DIG-CAR-Contract.pdf',
'http://www.bp.com/content/dam/bp-country/en_az/pdf/legalagreements/SHA_eng_intergov_Azerbaijan_Turkey_Intergovernmental_Agreement.pdf'
                      ]